# WAVE Client Python-JavaScript Integration Example

This notebook demonstrates the complete workflow of using both Python and JavaScript WAVE clients:

1. **Python Setup**: Create experiment and experiment type using Python client
2. **JavaScript Data Collection**: Generate an HTML page that uses the JavaScript client to log data
3. **Python Analysis**: Retrieve and analyze the data collected via JavaScript
4. **Cleanup**: Remove the test data

## Prerequisites

1. **WAVE Backend**: Running at `http://localhost:8000`
2. **Dependencies**: Run `make setup-local-dev` and select the python kernel in `.venv`
3. **API Key**: Set `WAVE_API_KEY` in your `.env` file
4. **Web Server**: You'll need to serve the generated HTML file via HTTP (not file://)

## Setup and Imports

In [ ]:
import os
import pandas as pd
from datetime import datetime
from dotenv import load_dotenv
from wave_client import WaveClient
from urllib.parse import quote
import webbrowser
from pathlib import Path

# Load environment variables
load_dotenv()
API_KEY = os.getenv("WAVE_API_KEY")
BASE_URL = os.getenv("WAVE_API_URL", "http://localhost:8000")

if not API_KEY:
    raise ValueError("Please set WAVE_API_KEY in your .env file")

print(f"✓ Connecting to {BASE_URL}")
print(f"✓ API Key loaded (ending in: ...{API_KEY[-4:]})")

✓ Connecting to http://localhost:8000
✓ API Key loaded (ending in: ...Pcv3KB)


In [2]:
# Initialize client and test connection
client = WaveClient(api_key=API_KEY, base_url=BASE_URL)

# Test connection
health = await client.get_health()
version = await client.get_version()

print(f"✓ Backend status: {health['status']}")
print(f"✓ API version: {version.get('api_version', 'unknown')}")

✓ Backend status: healthy
✓ API version: 1.0.0


## Step 1: Create Experiment Setup (Python)

In [3]:
# Create a unique experiment type for this demo
demo_name = f"JS Integration Demo {datetime.now().strftime('%Y%m%d_%H%M%S')}"
table_name = f"js_demo_{datetime.now().strftime('%Y%m%d_%H%M%S')}"

try:
    exp_type = await client.experiment_types.create(
        name=demo_name,
        table_name=table_name,
        description="Integration demo between Python and JavaScript clients",
        schema_definition={
            "trial_number": "INTEGER",
            "reaction_time": "FLOAT",
            "stimulus_color": "STRING",
            "response_key": "STRING",
            "accuracy": "BOOLEAN",
            "timestamp": "STRING",
            "user_agent": "STRING",
        },
    )
    exp_type_id = exp_type["id"]
    print(f"✓ Created experiment type: {exp_type['name']} (ID: {exp_type_id})")
    print(f"✓ Table name: {exp_type['table_name']}")
except Exception as e:
    print(f"Error creating experiment type: {e}")
    raise

✓ Created experiment type: JS Integration Demo 20250806_214923 (ID: 9)
✓ Table name: js_demo_20250806_214923


In [4]:
# Create the experiment
try:
    experiment = await client.experiments.create(
        experiment_type_id=exp_type_id,
        description=f"JavaScript integration demo - {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}",
        tags=[],
        additional_data={
            "created_by": "python_notebook",
            "integration_demo": True,
            "demo_type": "python_js_integration",
        },
    )
    experiment_id = str(experiment["uuid"])
    print(f"✓ Created experiment: {experiment['description']}")
    print(f"✓ Experiment ID: {experiment_id}")
except Exception as e:
    print(f"Error creating experiment: {e}")
    raise

✓ Created experiment: JavaScript integration demo - 2025-08-06 21:49:23
✓ Experiment ID: 461b7ebc-4457-4f35-a4fe-1f301d98e8f6


## Step 2: Generate Interactive HTML Page (JavaScript Client)

In [5]:
# Reference the existing JavaScript example (already modified to support our integration)
# Use pathlib for cleaner path handling
current_file = Path(__file__) if "__file__" in globals() else Path.cwd() / "dummy.py"
js_example_path = (
    current_file.parent.parent.parent / "javascript" / "examples" / "simple-experiment.html"
)

# Check if file exists
if not js_example_path.exists():
    # Try alternative path resolution
    current_dir = Path.cwd()
    alt_path = current_dir / "javascript" / "examples" / "simple-experiment.html"
    if alt_path.exists():
        js_example_path = alt_path
    else:
        print(f"❌ Could not find simple-experiment.html")
        print(f"   Tried: {js_example_path}")
        print(f"   Also tried: {alt_path}")
        print(f"   Current directory: {current_dir}")
        raise FileNotFoundError("JavaScript example file not found")

print(f"✓ Using JavaScript example at: {js_example_path}")
print(f"✓ File exists: {js_example_path.exists()}")
print(f"✓ The example has been pre-modified to support:")
print(f"  • URL parameter extraction for experiment_id and participant_id")
print(f"  • Enhanced data logging with all required schema fields")
print(f"  • Integration-ready configuration")

✓ Using JavaScript example at: /Users/douglas.wong1/code/wave-lab/wave-client/javascript/examples/simple-experiment.html
✓ File exists: True
✓ The example has been pre-modified to support:
  • URL parameter extraction for experiment_id and participant_id
  • Enhanced data logging with all required schema fields
  • Integration-ready configuration


In [6]:
# Generate the URL with all required parameters
# Generate a sample participant ID for the demo
sample_participant_id = f"demo_participant_{datetime.now().strftime('%H%M%S')}"

# Get absolute path
abs_path = js_example_path.resolve()

# Start a local HTTP server to avoid CORS issues with file:// URLs
import http.server
import socketserver
import threading
import time

# Find the project root (where javascript/ folder is)
project_root = abs_path.parent.parent.parent
print(f"📁 Project root: {project_root}")

# Start HTTP server in a separate thread
PORT = 8080
server_dir = project_root

class Handler(http.server.SimpleHTTPRequestHandler):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, directory=server_dir, **kwargs)

def start_server():
    with socketserver.TCPServer(("", PORT), Handler) as httpd:
        print(f"🌐 HTTP Server started at http://localhost:{PORT}")
        httpd.serve_forever()

# Start server in background thread
server_thread = threading.Thread(target=start_server, daemon=True)
server_thread.start()
time.sleep(1)  # Give server time to start

# Create HTTP URL instead of file URL
http_url = f"http://localhost:{PORT}/javascript/examples/simple-experiment.html"

# Create full URL with all required parameters
full_url = (
    f"{http_url}"
    f"?key={quote(API_KEY)}"
    f"&experiment_id={quote(experiment_id)}"
    f"&participant_id={quote(sample_participant_id)}"
)

censored_url = full_url.replace(API_KEY, "[HIDDEN]")

print("🌐 EXPERIMENT READY!")
print("=" * 50)
print(f"🌐 HTTP Server: http://localhost:{PORT}")
print(f"📁 HTML File: {abs_path}")
print(f"🔗 Full URL: {censored_url}")
print("\n📋 INSTRUCTIONS:")
print("1. Click the link below or copy the URL above")
print("2. Complete the reaction time task (5 trials)")
print("3. Return here to analyze the data")
print("\n💡 ABOUT THIS EXPERIMENT:")
print("• Uses the JavaScript example served over HTTP (avoids CORS issues)")
print("• URL parameters provide experiment_id, participant_id, and API key")
print("• Data automatically logged with our experiment schema")
print("• Simple click-based reaction time task")

# Open in browser with HTTP URL
print(f"\n🚀 Opening experiment in browser...")
try:
    webbrowser.open(full_url)
    print("✓ Browser opened with experiment ready to run.")
    print("💡 Server will run in background - experiment should work now!")
except Exception as e:
    print(f"❌ Could not open browser: {e}")
    print(f"📋 Manual steps:")
    print(f"   1. Open your browser")
    print(f"   2. Navigate to: {censored_url}")
    print(f"      (replace [HIDDEN] with your actual API key)")

📁 Project root: /Users/douglas.wong1/code/wave-lab/wave-client
🌐 HTTP Server started at http://localhost:8080
🌐 EXPERIMENT READY!
🌐 HTTP Server: http://localhost:8080
📁 HTML File: /Users/douglas.wong1/code/wave-lab/wave-client/javascript/examples/simple-experiment.html
🔗 Full URL: http://localhost:8080/javascript/examples/simple-experiment.html?key=[HIDDEN]&experiment_id=461b7ebc-4457-4f35-a4fe-1f301d98e8f6&participant_id=demo_participant_214923

📋 INSTRUCTIONS:
1. Click the link below or copy the URL above
2. Complete the reaction time task (5 trials)
3. Return here to analyze the data

💡 ABOUT THIS EXPERIMENT:
• Uses the JavaScript example served over HTTP (avoids CORS issues)
• URL parameters provide experiment_id, participant_id, and API key
• Data automatically logged with our experiment schema
• Simple click-based reaction time task

🚀 Opening experiment in browser...
✓ Browser opened with experiment ready to run.
💡 Server will run in background - experiment should work now!
🌐 EX

127.0.0.1 - - [06/Aug/2025 21:49:25] "GET /javascript/examples/simple-experiment.html?key=3ZVAv7r7G11UVMjmPuPcv3KB&experiment_id=461b7ebc-4457-4f35-a4fe-1f301d98e8f6&participant_id=demo_participant_214923 HTTP/1.1" 200 -
127.0.0.1 - - [06/Aug/2025 21:49:25] "GET /javascript/src/wave-client.js HTTP/1.1" 200 -
127.0.0.1 - - [06/Aug/2025 21:49:25] "GET /javascript/src/core/errors.js HTTP/1.1" 200 -
127.0.0.1 - - [06/Aug/2025 21:49:25] code 404, message File not found
127.0.0.1 - - [06/Aug/2025 21:49:25] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [06/Aug/2025 21:49:25] "GET /javascript/src/wave-client.js HTTP/1.1" 200 -
127.0.0.1 - - [06/Aug/2025 21:49:25] "GET /javascript/src/core/errors.js HTTP/1.1" 200 -
127.0.0.1 - - [06/Aug/2025 21:49:25] code 404, message File not found
127.0.0.1 - - [06/Aug/2025 21:49:25] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [06/Aug/2025 21:49:32] code 404, message File not found
127.0.0.1 - - [06/Aug/2025 21:49:32] "GET /.well-known/appspecific/com.

## Step 3: Monitor Data Collection

**⏸️ PAUSE HERE**: Complete the web experiment before continuing

Run the cell below to check for newly collected data:

In [7]:
# Check for data periodically
print("🔍 Checking for experiment data...")
print(f"📊 Experiment ID: {experiment_id}")

try:
    # Get all data for this experiment
    data_df = await client.experiment_data.get_all_data(experiment_id=experiment_id)

    if len(data_df) > 0:
        print(f"✅ Found {len(data_df)} data points!")
        print(f"📈 Participants: {data_df['participant_id'].nunique()}")
        print(
            f"📈 Latest trial: {data_df['trial_number'].max() if 'trial_number' in data_df.columns else 'N/A'}"
        )

        # Show basic info about the data
        print("\n📋 Data Summary:")
        print(
            data_df.groupby("participant_id")
            .agg({"trial_number": "count", "reaction_time": "mean", "accuracy": "mean"})
            .round(3)
        )

    else:
        print("⏳ No data found yet. Complete the web experiment first.")
        print("   Then run this cell again to check for data.")

except Exception as e:
    print(f"❌ Error retrieving data: {e}")

🔍 Checking for experiment data...
📊 Experiment ID: 461b7ebc-4457-4f35-a4fe-1f301d98e8f6
⏳ No data found yet. Complete the web experiment first.
   Then run this cell again to check for data.


## Step 4: Cleanup

In [8]:
# Set to True if you want to execute the cleanup process
cleanup_confirmed = False

if cleanup_confirmed:
    # Clean up the demo data
    print("🧹 Cleaning up demo data...")
    print("⚠️  This will delete the experiment and all associated data.")
    try:
        # Get all data for this experiment to delete
        all_data = await client.experiment_data.get_all_data(experiment_id)

        # Delete all experiment data first
        if len(all_data) > 0:
            print(f"Deleting {len(all_data)} data points...")
            deleted_count = 0

            for _, row in all_data.iterrows():
                try:
                    await client.experiment_data.delete_row(experiment_id, int(row["id"]))
                    deleted_count += 1
                except Exception as e:
                    print(f"  Failed to delete row {row['id']}: {e}")

            print(f"✓ Deleted {deleted_count} data rows")
        else:
            print("ℹ️  No experiment data found to delete")

        # Delete the experiment
        delete_response = await client.experiments.delete(experiment_id)
        print(f"✓ Deleted experiment: {experiment_id}")

        # Delete the experiment type
        delete_type_response = await client.experiment_types.delete(exp_type_id)
        print(f"✓ Deleted experiment type: {exp_type_id}")

        print("\n✅ Cleanup complete!")

    except Exception as e:
        print(f"❌ Error during cleanup: {e}")
        print("You may need to manually delete the experiment data.")

else:
    print("🔒 Automatic cleanup disabled.")
    print("\n📋 Manual cleanup instructions:")
    print(f"   1. Experiment ID to delete: {experiment_id}")
    print(f"   2. Experiment Type ID to delete: {exp_type_id}")
    print("\n💡 To enable automatic cleanup, set cleanup_confirmed = True above")

🔒 Automatic cleanup disabled.

📋 Manual cleanup instructions:
   1. Experiment ID to delete: 461b7ebc-4457-4f35-a4fe-1f301d98e8f6
   2. Experiment Type ID to delete: 9

💡 To enable automatic cleanup, set cleanup_confirmed = True above
